# Single Heart Rate Monitor Example

This is a minimal demonstration of how to set up and use Bleak-FSM.

This examples uses Bluetooth Low Energy (BLE) Heart Rate service.
+ It must support Bluetooth, not just ANT+
+ The GATT Characteristic required is: `00002a37-0000-1000-8000-00805f9b34fb`
+ You may use the [`bluetooth_dissect.py` tool](https://github.com/tensorturtle/bluetooth_dissect) to check if your device supports that GATT Characteristic.

Example compatible devices:
+ Magene H603
+ Wahoo TICKR & TICKR Fit
+ WHOOP 4.0 (Broadcast Heart Rate ON)
+ and many more advertised as "supports Bluetooth Heart Rate"

If you don't have access to a BLE Heart Rate device or want to a different BLE GATT characteristic, see the [Migration Guide](migration_guide.ipynb)


## How to run this Notebook

+ Make sure your computer's bluetooth adapter is powered on.
+ At first, I recommend stepping through each cell, reading the code and explanations.
+ You will need to set your Heart Rate monitor's address the first time.
+ At any time, `print(model.state)` to confirm which state you're in. This will help you figure out what you can do from there.

# Setup

In [1]:
import asyncio

In [2]:
from bleak_fsm import machine, BleakModel

In [3]:
model = BleakModel(connection_timeout=20)
machine.add_model(model)

Toggle this `USE_PYCYCLING` flag to see how this library can accomodate using standard, raw BLeakClient or Pycycling-wrapped BleakClient.

In [4]:
USE_PYCYCLING = True

In [5]:
# Define callbacks

if USE_PYCYCLING:
    from pycycling.heart_rate_service import HeartRateService
    model.wrap = lambda client: HeartRateService(client)
    model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
    model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
    def handle_hr_measurement(value):
        print("Using Pycycling wrapper around BleakClient")
        print(f"Heart Rate: {value}")
    model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)
else:
    HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"
    model.enable_notifications = lambda client: client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)
    model.disable_notifications = lambda client: client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)
    def handle_hr_measurement(sender, data):
        print("Using raw BleakClient")
        print(f"Data received from {sender}: {data}")
        flag = data[0]
        heart_rate = data[1]
        print(f"Heart Rate: {heart_rate} beats per minute")
    model.set_measurement_handler = lambda client: handle_hr_measurement


# Usage

Since we're in a Jupyter notebook, we need to use `create_task()` instead of `get_event_loop().run_until_complete()` to use the existing event loop which Jupyter runs on.

The `BleakModel` class represents this system's bluetooth adapter. This library currently supports one bluetooth adapter. We call the `start_scan()` class method.

In [6]:
await BleakModel.start_scan()

True

In [7]:
async def print_as_we_go(max_devices=30):
    for i in range(40):
        if len(BleakModel.bt_devices) >= max_devices:
            print(f"{len(BleakModel.bt_devices)} devices found. Stopping scan.")
            await model.stop_scan()
            break
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

0 devices found in 0.0 seconds.
3 devices found in 0.05 seconds.
5 devices found in 0.1 seconds.
7 devices found in 0.15 seconds.


7 devices found in 0.2 seconds.
7 devices found in 0.25 seconds.
7 devices found in 0.3 seconds.
7 devices found in 0.35 seconds.
8 devices found in 0.4 seconds.
8 devices found in 0.45 seconds.
9 devices found in 0.5 seconds.
9 devices found in 0.55 seconds.
10 devices found in 0.6 seconds.
10 devices found in 0.65 seconds.
10 devices found in 0.7 seconds.
10 devices found in 0.75 seconds.
10 devices found in 0.8 seconds.
10 devices found in 0.85 seconds.
10 devices found in 0.9 seconds.
10 devices found in 0.95 seconds.
10 devices found in 1.0 seconds.
10 devices found in 1.05 seconds.
10 devices found in 1.1 seconds.
10 devices found in 1.15 seconds.
10 devices found in 1.2 seconds.
10 devices found in 1.25 seconds.
10 devices found in 1.3 seconds.
10 devices found in 1.35 seconds.
11 devices found in 1.4 seconds.
11 devices found in 1.45 seconds.
11 devices found in 1.5 seconds.
11 devices found in 1.55 seconds.
11 devices found in 1.6 seconds.
11 devices found in 1.65 seconds.
11 

In [8]:
await BleakModel.stop_scan()

True

In [9]:
BleakModel.bt_devices

{'14863C4D-BF71-4EA3-C6B4-98001056AAF8': (BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
  AdvertisementData(local_name='WHOOPDEDOO', service_uuids=['0000180d-0000-1000-8000-00805f9b34fb'], rssi=-53)),
 'DBAC2719-6832-42A9-CC67-7D2997A31AEE': (BLEDevice(DBAC2719-6832-42A9-CC67-7D2997A31AEE, None),
  AdvertisementData(manufacturer_data={76: b'\t\x08\x13\xc3\xac\x1e\x01Q\x1bX\x13\x08\n\xd6\xa3\x19\x9e\xe82\x00'}, rssi=-40)),
 '3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862': (BLEDevice(3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862, Green Apple),
  AdvertisementData(manufacturer_data={76: b'\x10\x074\x1fM\xa4\n\x08X'}, tx_power=7, rssi=-40)),
 'DC2343A7-C46E-E1BB-30E9-BC6E274B4A39': (BLEDevice(DC2343A7-C46E-E1BB-30E9-BC6E274B4A39, Fast Fred),
  AdvertisementData(manufacturer_data={76: b'\x10\x06\x08\x1dA\x9ex\x08'}, tx_power=12, rssi=-48)),
 'AE88AF87-8B7B-E79C-B10A-5ADEE088AD79': (BLEDevice(AE88AF87-8B7B-E79C-B10A-5ADEE088AD79, None),
  AdvertisementData(manufacturer_data={76: b'\x16\x08\x00\

Copy-and-paste the key corresponding to the device that you want to connect to:

In [10]:
target_address = "EF:E4:F6:7D:6C:F6"

Or search for the name of the device from the values and retrieve its key.

While most consumer heart rate monitors have a name, this name might be not unique or missing.

In [11]:
target_name = "WHOOPDEDOO"

target_address = ""
for address, (ble_device, advertisement_data) in BleakModel.bt_devices.items():
    if ble_device.name == target_name:
        target_address = address
        print(f"Found {target_name} at {target_address}")

Found WHOOPDEDOO at 14863C4D-BF71-4EA3-C6B4-98001056AAF8


Anyhow, we will attempt to connect to the following address:

In [12]:
print(target_address)

14863C4D-BF71-4EA3-C6B4-98001056AAF8


In [13]:
await model.set_target(target_address)

True

 When you provide a target address to the `model`, its state changes to `TargetSet`

In [14]:
model.state

'TargetSet'

From `TargetSet` state, you may now attempt to connect to the device.

By default, if it has been over 120 seconds since the last scan, if `auto_rescan=True`, the model will automatically re-scan and then attempt the connection.
These parameters may be changed in the `BleakModule` instantiation above.

In [15]:
success = await model.connect()

if success:
    print("Connected to device")

Connected to device


Sidebar: Note how the above cell returns `True`. All transition methods on `model` returns True if the transition was successful. Therefore you may explicitly define happy-path and exception-path in different ways.

For debugging,
```python
assert await model.connect(), "Model failed to connect"
```

Or, more properly check the return value
```python
success = await model.connect()
if not succcess:
    await model.clean_up() # this catch-all method brings the model back to "Init"
    raise Exception("Failed to connect")
```

Or, we can check the `.state` afterwards, like so:

In [16]:
if model.state != "Connected":
    raise Exception("Failed to connect. If the device was not properly disconnected last time, restart either the device or bluetooth adapter")

In order to receive/send actual data from/to the connected device, we need to start a stream.

This will run forever until `model.disconnect()` is called, so for now let's sleep for several seconds and then call that.

In [17]:
await model.stream()

True

In [18]:
await asyncio.sleep(5)

Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[893], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=0, rr_interval=[0, 0, 0, 0], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=67, rr_interval=[829], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=0, rr_interval=[0, 0, 0, 0], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=67, rr_interval=[869], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=0, rr_interval=[0, 0, 0, 0], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bp

Bleak-FSM encapsulates a kind of bug in Bleak, where it is not possible to re-use a connected client. 

That is, we shouldn't step back to "Connected" state from "Streaming", because we can't go from "Connected" back to "Streaming".

Instead we need to go back two steps to "TargetSet", re-establish a connection, and then we can stream again.

Bleak-FSM enforces this behavior for your own safety.
There exists not method for you to even try to go from "Streaming" to "Connected".


In [19]:
await model.disconnect()

True

Whenever the application terminates, or otherwise no longer needs to communicate with the bluetooth device, it should always call `disconnect()` on the model. The model state should always e either "TargetSet" or "Init".

In [20]:
model.state

'TargetSet'

In [21]:
await model.unset_target()

True

In [22]:
model.state

'Init'

For convenience, you may call `await model.clean_up()` from any state to come back to "Init"

In [23]:
await model.clean_up()

True

For even more convenience, you may call the class's `clean_up_all()` classmethod to disconnect all devices:

In [34]:
await BleakModel.clean_up_all()

True

From "Scanning", transitioning to any other state besides "Init" is illegal: